# matchup code
this code reformats the GNATS DateTime stamp so that it matches the satellite data format and then searches the satellite file names to find which ones are closest in time

In [2]:
#numeric python module, useful for handling large data arrays
import numpy as np

import pandas as pd

#module for dealing with netCDF files (all sat files are netCDF)
import netCDF4 as nc

#modules for making plots. Here wer are importing the full matplotlib
#module, but also the sub-module pyplot separately with an abbreviated
#name so it is easier to use in the code
import matplotlib.pyplot as plt
import matplotlib

#panda
import pandas as pd


#import datetime
import datetime
from datetime import datetime

#for displaying the plots in this notebook 
%matplotlib inline

#module for making maps
from mpl_toolkits.basemap import Basemap

#miscellaneous operating system interfaces module for doing things like
#moving file, exploring directory paths, etc..
import os
import re


In [3]:
##THIS IS FOR THE GNATS
#specify the directory where all the files are saved
cdom_gnatsdir= '/Users/clarabirdferrer/Documents/Bigelow/Data/GNATS'


wd = os.chdir(cdom_gnatsdir)
cwd = os.getcwd()
#print (cwd)

from pandas import DatetimeIndex

##GNATS shipboard data
#1.import txt file
fn= 'GNATS_date_cdom.txt'
df_gn = pd.read_csv('GNATS_date_latlon_cdom.csv',sep=",",header = (1), names=['date','lon','lat','cdom'])
#print(df)

#2.convert all -999 cdom data points into nan
ag = df_gn['cdom']
ag[ag < 0] = np.nan
#print(df)

#3.convert date to satellite format
##satelite = YYYYMMDOYHHMMSS
##GNATS = YYYY-MM-DDTHH:MM:SS
#print(df_gn['date'])
date = df_gn['date']
SatDateTime = []
datetime = []
for i in date:
    YYYY = i[:4]  #this isolates the 4 digits for YYYY
    
    month = i[5:]  #this isolates the digits after year but there is extra
    mm = month[:2] #this isolates just MM
    day = i[8:]    #this isolates the digits after month
    dd = day[:2]   #this isoaltes just DD
   
    
    ##attempt 1 at combining month and day to form DOY
    #this converts months to julian day months
    if mm == '01':
        M = '001'
    elif mm == '02':
        M = '032'
    elif mm == '03':
        M = '060'
    elif mm == '04':
        M = '091'
    elif mm == '05':
        M = '121'
    elif mm == '06':
        M = '152'
    elif mm == '07':
        M = '182'
    elif mm == '08':
        M = '213'
    elif mm == '09':
        M = '244'
    elif mm == '10':
        M = '274'
    elif mm == '11':
        M = '305'
    elif mm == '12':
        M = '335'
    
    #this takes care of the leap year thing
    if YYYY == '2004':
        if M == '001':
            M = '001'
        elif M == '032':
            M = '032'
        elif M == '060':
            M = '061'
        elif M == '091':
            M = '092'
        elif M == '121':
            M = '122'
        elif M == '152':
            M = '153'
        elif M == '182':
            M = '183'
        elif M == '213':
            M = '214'
        elif M == '244':
            M = '245'
        elif M == '274':
            M = '275'
        elif M == '305':
            M = '306'
        elif M == '335':
            M = '336'      
    elif YYYY == '2008':
        if M == '001':
            M = '001'
        elif M == '032':
            M = '032'
        elif M == '060':
            M = '061'
        elif M == '091':
            M = '092'
        elif M == '121':
            M = '122'
        elif M == '152':
            M = '153'
        elif M == '182':
            M = '183'
        elif M == '213':
            M = '214'
        elif M == '244':
            M = '245'
        elif M == '274':
            M = '275'
        elif M == '305':
            M = '306'
        elif M == '335':
            M = '336'
    elif YYYY == '2012':
        if M == '001':
            M = '001'
        elif M == '032':
            M = '032'
        elif M == '060':
            M = '061'
        elif M == '091':
            M = '092'
        elif M == '121':
            M = '122'
        elif M == '152':
            M = '153'
        elif M == '182':
            M = '183'
        elif M == '213':
            M = '214'
        elif M == '244':
            M = '245'
        elif M == '274':
            M = '275'
        elif M == '305':
            M = '306'
        elif M == '335':
            M = '336'
    elif YYYY == '2016':
        if M == '001':
            M = '001'
        elif M == '032':
            M = '032'
        elif M == '060':
            M = '061'
        elif M == '091':
            M = '092'
        elif M == '121':
            M = '122'
        elif M == '152':
            M = '153'
        elif M == '182':
            M = '183'
        elif M == '213':
            M = '214'
        elif M == '244':
            M = '245'
        elif M == '274':
            M = '275'
        elif M == '305':
            M = '306'
        elif M == '335':
            M = '336'
    
    #this makes the julian day month values and the day values into int so we can add them
    #then it adds them together to get the day of the year (DOY)
    M=int(M)
    dd=int(dd)
    DOY = M + dd - 1
    DOY = str(DOY)  ##make string again so you can add the string to YYYY instead of adding values
    dd = str(dd)    ##Put dd back to string 
    if len(dd) == 1:    ##int takes away the zero for dd like 07 -> 7
        dd = '0' + dd   ##these two lines put the 0 back -> its import when converting to datetime later
    #print(dd)
    
    #this is just to make sure we're on the right track by combining year and DOY
    #print(YYYY + DOY) #yay it works!
    
    #ok so now we have to continue isolating factors -> we still need HHMMSS
    hour = i[11:] ##isolates the digits starting with hour
    H = hour[:2] #isolates just HH
    minute = i[14:] 
    MM = minute[:2]  #isolates just the MM
    SS = i[17:]      #isolates just the SS
    
    ##this converts local time to GMT
    year = int(YYYY)
    mon = int(mm)
    day = int(dd)
    #print(mon)
    h = int(H)
    if year <= 2006:
        if mon == 4:      ##this mess is because daylights savings is part way through the month
            if year == 2006:  ##the exact day is different based on year
                if day >=2:
                    HH = h + 4
        elif mon == 10:
            if year == 2003:
                if day < 26:
                    HH = h + 4
            elif year == 2005:
                if day < 30:
                    HH = h + 4
        elif mon == 5:
            HH = h + 4
        elif mon == 6:
            HH = h + 4
        elif mon == 7:
            HH = h + 4
        elif mon == 8:
            HH = h + 4
        elif mon == 9:
            HH = h + 4
        elif mon == 11:
            HH = h + 5
        elif mon == 12:
            HH = h + 5
        elif mon == 1:
            HH = h + 5
        elif mon == 2:
            HH = h + 5
        elif mon == 3:
            HH = h + 5
    elif year >= 2007:     ##the day lights savings months changed in 2007
        if mon == 3:
            if year == 2007:
                if day >= 11:
                    HH = h + 4
            elif year == 2010:
                if day < 14:
                    HH = h + 5
            elif year == 2013:
                if day < 10:
                    HH = h + 5
        elif mon == 11:
            if year == 2008:
                if day >= 2:
                    HH = h + 5
            elif year == 2009:
                if day >= 1:
                    HH = h + 5
            elif year == 2012:
                if day >= 4:
                    HH = h + 5
        elif mon == 4:
            HH = h + 4
        elif mon == 5:
            HH = h + 4
        elif mon == 6:
            HH = h + 4
        elif mon == 7:
            HH = h + 4
        elif mon == 8:
            HH = h + 4
        elif mon == 9:
            HH = h + 4
        elif mon == 10:
            HH = h + 4
        elif mon == 12:
            HH = h + 5
        elif mon == 1:
            HH = h + 5
        elif mon == 2:
            HH = h + 5
    #print(HH)    
    
    HH = str(HH)
    if len(HH) == 1:
        HH = '0' + HH
    
    #this takes all the components we've isolated and put them together
    sat_time = YYYY + DOY + HH + MM + SS
    #print(sat_time)
    SatDateTime += [sat_time]

    #this puts together typical datetime YYYYMMDDHHMMSS format so can convert to datetime later on
    DateTime = YYYY + mm + dd + HH + MM + SS
    datetime += [DateTime]
    
    
#this adds column to gnats df with satellite format date time
df_gn['satdate'] = SatDateTime
df_gn = df_gn.sort_values(by='date')

#this adds column with usual date time format column
#datetime = str(datetime)
df_gn['datetime'] = datetime
#print(df_gn)
    

/Users/clarabirdferrer/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
#search satellite file names for times that match up to GNATS
#import the list of satellite file names
df_sat = pd.read_csv('GNATS_satellite_files.csv',sep=",",header = (-1), names=['SatFile'])
#print(df_sat)

sat_dates = []
for i in df_sat['SatFile']:
    dateA = i[:8]               ##this isolates only YYYYDOY because the exact time is too specific and won't match up
    date_sat = dateA[1:]        ##you could expand this to include hour
    sat_dates += [date_sat] 
df_sat['matchdate'] = sat_dates
#print(df_sat['SatFile'])

gnats_dates = []
for ii in df_gn['satdate']:
    date_gnats = ii[:7]
    gnats_dates += [date_gnats]
df_gn['matchdate'] = gnats_dates
#print(df_gn)    

##identifies sat file names that match up with gn

sat_matchup = df_sat.loc[df_sat['matchdate'].isin(df_gn['matchdate'])]
gnats_matchup = df_gn.loc[df_gn['matchdate'].isin(df_sat['matchdate'])]
#print(sat_matchup)
#print(gnats_matchup)

In [5]:
##convert sat dates to YYYYMMDDHHMMSS -> need this to convert to dt.datetime
sat_datetime = []
import datetime as dt
from datetime import datetime
for i in sat_matchup['SatFile']:     ##this is separating out the julian date
    dateA = i[:8]
    date = dateA[1:]
    month = date[4:]
    year = date[:4]
    juldate = year + month
    time = i[8:]
      
    date = dt.datetime.strptime(juldate,'%Y%j').date()  ##converts julian date to normal date
    date = str(date)
    yr = date[:4]
    #print(yr)
    mo = date[5:]
    mo = mo[:2]
    #print(mo)
    dy = date[8:]
    #print(dy)
    #print(yr + mo + dy)
    satdate = yr + mo + dy + time
    
    sat_datetime += [satdate]
sat_matchup['DateTime'] = sat_datetime  
#print(sat_matchup['DateTime'])

/Users/clarabirdferrer/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
##convert both GNATS and satfile to actual DateTime so can use to subtract
import datetime as dt
from datetime import datetime, timedelta
#print(df_gn['datetime'])
#dt.datetime.strptime(df_sat['DateTime'],'%Y%m%d%H%M%S')
#df_gn['datetime'] = pd.to_datetime(df_gn['datetime'], format = '%Y%m%d%H%M%S',error='coerce')

#convert gnats to datetime
gnats_matchup['datetime'] = gnats_matchup['datetime'].convert_objects(convert_dates='coerce')
gnats_matchup = gnats_matchup.sort_values(by='datetime')
pd.options.display.max_rows = 10000
#print(gnats_matchup['datetime'])

#print(sat_matchup['DateTime'])
#convert sat date to datetime
sat_matchup['DateTime'] = sat_matchup['DateTime'].convert_objects(convert_dates='coerce')

#print(sat_matchup['DateTime'])

/Users/clarabirdferrer/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
/Users/clarabirdferrer/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/clarabirdferrer/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()
/Users/clarabirdferrer/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A va

In [7]:
#### calculate difference in hours and exclude if >3 hour difference

#satellite files
sat_within_time = []
gnats_within_time = []
gnats_L2filename = []
for ii in sat_matchup['DateTime']:
    time_diff = abs(ii - gnats_matchup['datetime'])
    time_diff = str(time_diff)
    #print(time_diff)
    
    timelimit = dt.time(3)
    timelimit = str(timelimit)
    #print(timelimit.dtype)
    if time_diff <= timelimit:
        iii = str(ii)
        sat_within_time += [iii]
        

d={'A':sat_within_time}
df_sat_within = pd.DataFrame(data=d)
#print(df_within)

sat_files_for_matchup = sat_matchup.loc[sat_matchup['DateTime'].isin(df_sat_within['A'])]
#print(sat_files_for_matchup)

#save satellite file names to csv file
header = ["SatFile"]
sat_files_for_matchup.to_csv('sat_files_matchup.csv',columns=header)


In [ ]:
##this creates array of corresponding L2 file for gnats file
import operator
gnatsSatFileMatch = [None]*len(gnats_matchup['datetime'])
for i,gnats_dt in enumerate(gnats_matchup['datetime']):   ##i is index
    print("line"+ " " + str(gnats_dt))
    for (satDT,satfile) in zip(sat_matchup['DateTime'],sat_matchup['SatFile']): #run through L2files for each gnats pt
        time_diff = gnats_dt - satDT
        timeDiff = time_diff.days*24 + time_diff.seconds/60/60
        if abs(timeDiff) <= 3:          ##if time diff is less than 3 then add corresponding L2 to array
            ii = int(i)
            gnatsSatFileMatch[ii] = satfile
            break
            
gnats_matchup['L2file'] = gnatsSatFileMatch

#print(gnats_matchup)

In [ ]:
for i,ii,iii in zip(gnats_matchup['L2file'],gnats_matchup['datetime'],gnats_matchup['timediff']):
    if i != None:
        #print(i,ii,iii)
#print(timediff)

In [ ]:
L2 = []
for i in gnats_matchup['L2file']:
    if i!= None:     ##this is getting rid of the None values in the array
        L2 += [i]
            
               
d = {'A':L2}        ##make single column dataframe for search and replace
dL = pd.DataFrame(data=d)
CDOM_match = gnats_matchup.loc[gnats_matchup['L2file'].isin(dL['A'])]

cdom_stddev = [None]*len(CDOM_match['date'])  ##this adds column of all -99
for ii,cdom_dt in enumerate(CDOM_match['date']):
    cdom_stddev[ii] = -99

cdom_id = [None]*len(CDOM_match['date'])   ##this adds column that is just numbered
for i,cdomdt in enumerate(CDOM_match['date']):
    cdom_id[i] = i
    
CDOM_match['cdom_stddev'] = cdom_stddev   ##adds the columns to the df
CDOM_match['ID'] = cdom_id

CDOM_match = CDOM_match.dropna(axis=0, how='any') #gets rid of all the rows where CDOM = NaN

print(CDOM_match)

In [22]:
#this is saving file to csv but we only want some of the columns
header = ["ID","date","lat","lon","L2file","cdom","cdom_stddev"]
CDOM_match.to_csv('CDOM_gnats_sat_matchup.csv',columns=header)